In [2]:
import browser_cookie3
import myfitnesspal

# cookies z Firefoxa 
cj = browser_cookie3.firefox(domain_name='myfitnesspal.com')
client = myfitnesspal.Client(cookiejar=cj)

MyfitnesspalLoginError: Could not access MyFitnessPal using the cookies provided by your browser.  Are you sure you have logged in to MyFitnessPal using a browser on this computer?

In [3]:
import datetime

# Pobieramy dane dla wybranej daty, np. wczorajszej
yesterday = datetime.date.today() - datetime.timedelta(days=1)
day = client.get_date(yesterday.year, yesterday.month, yesterday.day)

# Podsumowanie makroskładników
print(f"Dnia {yesterday}:")
print(day.totals)
# => {'calories': 2000, 'carbohydrates': 250, 'fat': 70, 'protein': 100, ...}

Dnia 2025-04-25:
{'calories': 2365.0, 'carbohydrates': 346.0, 'fat': 55.0, 'protein': 140.0, 'sodium': 3128.0, 'sugar': 93.0}


In [4]:
for meal in day.meals:
    print("Posiłek:", meal.name)
    for entry in meal.entries:
        # entry.name to nazwa produktu, entry.totals to słownik makro
        print(f"  - {entry.name}: {entry.totals}")

Posiłek: breakfast
  - protein shake vanilla - protein shake vanilla, 300 ml: {'calories': 312.0, 'carbohydrates': 17.0, 'fat': 13.0, 'protein': 33.0, 'sodium': 0.0, 'sugar': 0.0}
Posiłek: lunch
  - Homemade - Pasta Carbonara, 2 portion: {'calories': 818.0, 'carbohydrates': 122.0, 'fat': 18.0, 'protein': 53.0, 'sodium': 1376.0, 'sugar': 16.0}
Posiłek: dinner
  - Corn flakes, 7 cup: {'calories': 625.0, 'carbohydrates': 147.0, 'fat': 1.0, 'protein': 13.0, 'sodium': 1276.0, 'sugar': 17.0}
  - Milk, reduced fat, 2%, 5 cup: {'calories': 610.0, 'carbohydrates': 60.0, 'fat': 23.0, 'protein': 41.0, 'sodium': 476.0, 'sugar': 60.0}
Posiłek: snacks


In [8]:
import datetime

# zakres dat
today = datetime.date.today()
start = today - datetime.timedelta(days=7)   # ostatni rok
dates = [start + datetime.timedelta(days=i) 
         for i in range((today - start).days + 1)]

# totals dla każdej daty
rows = []
for d in dates:
    day = client.get_date(d.year, d.month, d.day)
    data = {
        'date':         d.isoformat(),
        'calories':     day.totals.get('calories', 0),
        'carbohydrates':day.totals.get('carbohydrates', 0),
        'fat':          day.totals.get('fat', 0),
        'protein':      day.totals.get('protein', 0),
        # możesz dorzucić inne klucze np. 'sodium', 'fiber' jeżeli są
    }
    rows.append(data)

import json

with open('mfp_daily_totals.json', 'w', encoding='utf-8') as f:
    json.dump(rows, f, ensure_ascii=False, indent=2)

print("Zapisano mfp_daily_totals.json")

Zapisano mfp_daily_totals.json


In [7]:
import datetime
import json

today = datetime.date.today()
start = today - datetime.timedelta(days=7)
dates = [start + datetime.timedelta(days=i)
         for i in range((today - start).days + 1)]

data = []
for d in dates:
    day = client.get_date(d.year, d.month, d.day)
    day_dict = {
        'date':   d.isoformat(),
        'totals': day.totals,
        'meals':  []
    }

    for meal in day.meals:
        meal_dict = {
            'name':    meal.name,
            'entries': []
        }
        for entry in meal.entries:
            meal_dict['entries'].append({
                'food':      entry.name,
                # jeśli entry.serving_size istnieje, dorzuć ilość:
                **({'quantity': entry.serving_size} if hasattr(entry, 'serving_size') else {}),
                'totals':    entry.totals
            })
        day_dict['meals'].append(meal_dict)

    data.append(day_dict)


with open('mfp_daily_totals_and_menu.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Zapisano mfp_daily_totals_and_menu.json")

Zapisano mfp_daily_totals_and_menu.json


In [ ]:
import os
import json
import datetime
from dotenv import load_dotenv

from playwright.sync_api import sync_playwright
from requests.cookies import RequestsCookieJar
import myfitnesspal

def fetch_mfp_and_save():
    # 1. Wczytaj poświadczenia z .env
    load_dotenv()
    USER = os.getenv("MFP_USERNAME")
    PASS = os.getenv("MFP_PASSWORD")
    if not USER or not PASS:
        raise ValueError("Brakuje MFP_USERNAME lub MFP_PASSWORD w pliku .env")

    # 2. Odpalenie headless Chromium przez Playwright
    with sync_playwright() as pw:
        browser = pw.chromium.launch(headless=True)
        page    = browser.new_page()

        # 2a. Przejdź na stronę logowania
        page.goto("https://www.myfitnesspal.com/account/login")

        # 2b. Wypełnij formularz i zaloguj
        page.fill('input[name="email"]', USER)
        page.fill('input[name="password"]', PASS)
        page.click('button[type="submit"]')

        # 2c. Poczekaj aż strona się w pełni załaduje (brak ruchu sieciowego)
        page.wait_for_load_state("networkidle", timeout=20000)

        # 2d. Pobierz ciasteczka z sesji
        raw_cookies = page.context.cookies()
        browser.close()

    # 3. Konwertuj na RequestsCookieJar
    jar = RequestsCookieJar()
    for c in raw_cookies:
        jar.set(c["name"], c["value"], domain=c.get("domain"), path=c.get("path"))

    # 4. Stwórz klienta myfitnesspal z zalogowanymi ciasteczkami
    client = myfitnesspal.Client(cookiejar=jar)

    # 5. Pobierz dane z ostatnich 7 dni
    today = datetime.date.today()
    start = today - datetime.timedelta(days=7)
    dates = [start + datetime.timedelta(days=i) for i in range((today - start).days + 1)]

    data = []
    for d in dates:
        day = client.get_date(d.year, d.month, d.day)
        data.append({
            "date":   d.isoformat(),
            "totals": day.totals,
            "meals":  [
                {
                    "name":    meal.name,
                    "entries": [
                        {
                            "food":      entry.name,
                            **({"quantity": entry.serving_size}
                               if hasattr(entry, "serving_size") else {}),
                            "totals":    entry.totals
                        }
                        for entry in meal.entries
                    ]
                }
                for meal in day.meals
            ]
        })

    # 6. Zapisz do pliku JSON
    with open("mfp_daily_totals_and_menu.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print("✅ Zapisano mfp_daily_totals_and_menu.json")

if __name__ == "__main__":
    fetch_mfp_and_save()

TimeoutError: Page.wait_for_selector: Timeout 10000ms exceeded.
Call log:
  - waiting for locator("nav[aria-label=\"Top Navigation\"]") to be visible
